In [3]:
import netCDF4 as nc

grid2 = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')

grid2['e3w_0'][0,:,250,250]

masked_array(data=[ 1.        ,  1.00000277,  1.00000815,  1.00001865,
                    1.00003908,  1.00007889,  1.00015642,  1.00030743,
                    1.00060155,  1.00117439,  1.00229008,  1.00446285,
                    1.00869379,  1.01693058,  1.03295863,  1.06412017,
                    1.12459981,  1.24158994,  1.46643748,  1.89327204,
                    2.68485706,  4.0913131 ,  6.40988864,  9.79767798,
                   13.96660882, 18.13553966, 21.523329  , 18.8604349 ,
                   14.34398256, 26.0399456 , 26.46678016, 26.69162771,
                   26.80861783, 26.86909747, 26.90025901, 26.91628706,
                   26.92452385, 26.92875479, 26.93092757, 26.93204325],
             mask=False,
       fill_value=1e+20)

In [9]:
def yearved_de(spacing,stn_b,stn_e,year):
    
    import map_fxn as mf
    import time
    import xarray as xr
    import numpy as np
    print('Spacing between stations: ' + str(spacing))
    
    bath = '/results/nowcast-sys/NEMO-forcing/grid/mesh_mask_SalishSea2.nc'
    grid = mf.import_bathy(bath)
    fmask = (grid.fmask[0,0,:,:])
    
    stn_x, stn_y = mf.make_stns(spacing)
    d_stn_x, d_stn_y = mf.filter_stn_in_domain(stn_x,stn_y,fmask)
    
    no_stns = len(d_stn_x)
    monlist = ['jan','feb','mar','apr','may','jun','jul','aug','sep',
              'oct','nov','dec']
    if year == 2016: 

        daylist = [31,29,31,30,31,30,31,31,30,31,30,31]
        noday = 366
        print(noday)
    if year != 2016:
        daylist = [31,28,31,30,31,30,31,31,30,31,30,31]
        noday = 365
        print(noday)
        

    # a list of all the model outputs (tracers) for this year    
    trace_list = mf.create_physlist_W(year)
    print(trace_list)
    print("Number of stations:" + str(no_stns))
    
    for stn in range(stn_b,stn_e):
    
        print('station is: ' ,str(stn))
        print('x is :', d_stn_x[stn])
        print('y is :', d_stn_y[stn])
        
        ts_x = d_stn_x[stn]
        ts_y = d_stn_y[stn]
        
        daily_ved = np.zeros(noday)
        
        end_day = noday
        for day in range(1,2):#end_day+1):
            if (day%20 ==0) :
                print(day)
            trc = trace_list[day-1]
            print(trc)
            #print(trc)
            nemo = xr.open_mfdataset(trc)
            #print(nemo)
            ved = np.array(nemo.variables['vert_eddy_diff'])
            ved = np.squeeze(ved)
            
            depth = np.array(nemo.variables['depthw_bounds'])
            depth = depth[:,1]
            if day == 1:
                print(depth)
            ved_here = ved[:,ts_y,ts_x]
            #ved_here = np.squeeze(ved_here)
            where0 = np.where(ved_here == 0)
            if day == 1:
                print(where0)
            #where0 = np.squeeze(where0)
            first_oob = (where0[1])#surface is also always zero
            depth_here = depth[first_oob-1]
            if day == 1:
                print(depth_here)
            total_ved = np.sum(ved_here[0:first_oob])
            avg_ved = total_ved/depth_here
            
            daily_ved[day-1] = avg_ved
    
        ved = xr.Dataset({'daily_ved':(['t'], daily_ved)})
        
        dirname = '/data/tjarniko/MEOPAR/analysis-tereza/notebooks/CLUSTER_201905/NC_HINDCAST/' + str(year) +'/VED_TS/'
        stn_name = dirname + 'stn_' + str(stn)  + 'avg_ved_sp' + str(spacing)+ '.nc'
        
        ved.to_netcdf(stn_name)


In [11]:
yearved_de(10,0,1,2013)

Spacing between stations: 10
365
20130101
/data/tjarniko/avg/SalishSea_1d_20130101_20130101_grid_W.nc
20130102


IndexError: list index out of range

In [52]:
spacing = 10
noday = 365

import map_fxn as mf
import time
import netCDF4 as nc
import numpy as np

bath = '/results/nowcast-sys/NEMO-forcing/grid/mesh_mask_SalishSea2.nc'
grid = mf.import_bathy(bath)
#use grid slightly differently to get z-direction 
grid2 = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
fmask = (grid.fmask[0,0,:,:])

stn_x, stn_y = mf.make_stns(spacing)
d_stn_x, d_stn_y = mf.filter_stn_in_domain(stn_x,stn_y,fmask)


for stn in range(0,1):

    print('station is: ' ,str(stn))
    print('x is :', d_stn_x[stn])
    print('y is :', d_stn_y[stn])

    ts_x = d_stn_x[stn]
    ts_y = d_stn_y[stn]

    daily_ved = np.zeros(noday)

    end_day = noday
    for day in range(1,2):#end_day+1):
        if (day%20 ==0) :
            print(day)
        trc = trace_list[day-1]
        nemo = nc.Dataset(trc)
        #print(nemo)
        ved = np.array(nemo.variables['vert_eddy_diff'])
        ved  = w['vert_eddy_diff'][0,:,ts_y,ts_x]
        where0 = (np.where(ved == 0))
        #first out of bounds cell
        stop = where0[0][1]
        #get only applicable veds
        veds = ved[0:stop]
        #get applicable depths
        depths = grid2['e3w_0'][0,0:stop,ts_y,ts_x]

        #get total depth to fially divide by 
        totdepth = np.sum(depths)
        #weight veds by depths - ie size of cell
        veds_depths = veds*depths
        #get avg
        avg_ved = np.sum(veds_depths/totdepth)

        daily_ved[day-1] = avg_ved

    ved = xr.Dataset({'daily_ved':(['t'], daily_ved)})

    dirname = '/data/tjarniko/MEOPAR/analysis-tereza/notebooks/CLUSTER_201905/NC_HINDCAST/' + str(year) +'/VED_TS/'
    stn_name = dirname + 'stn_' + str(stn)  + 'avg_ved_sp' + str(spacing)+ '.nc'

    ved.to_netcdf(stn_name)

station is:  0
x is : 2
y is : 392


NameError: name 'stn_name' is not defined

In [51]:
w = nc.Dataset('/data/tjarniko/avg/SalishSea_1d_20130101_20130101_grid_W.nc')
grid2 = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
print(w)

ved  = w['vert_eddy_diff'][0,:,250,250]
where0 = (np.where(ved == 0))
#first out of bounds cell
stop = where0[0][1]
#get only applicable veds
veds = ved[0:stop]
#get applicable depths
depths = grid2['e3w_0'][0,0:stop,250,250]

#get total depth to fially divide by 
totdepth = np.sum(depths)
#weight veds by depths - ie size of cell
veds_depths = veds*depths
#get avg
avg_ved = np.sum(veds_depths/totdepth)
print(avg_ved)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: SalishSea_1h_20130101_20130105
    description: ocean W grid variables
    title: ocean W grid variables
    Conventions: CF-1.6
    timeStamp: 2019-Sep-14 19:05:52 GMT
    uuid: 46a011b2-205e-4e89-bfc7-7e4b804910bd
    history: Thu Aug 20 06:24:28 2020: ncra /results2/SalishSea/hindcast.201905/01jan13/SalishSea_1h_20130101_20130101_grid_W.nc /data/tjarniko/avg/SalishSea_1d_20130101_20130101_grid_W.nc
    NCO: "4.5.4"
    nco_openmp_thread_number: 1
    dimensions(sizes): y(898), x(398), nvertex(4), depthw(40), axis_nbounds(2), time_counter(1)
    variables(dimensions): float32 nav_lat(y,x), float32 nav_lon(y,x), float32 bounds_lon(y,x,nvertex), float32 bounds_lat(y,x,nvertex), float32 area(y,x), float32 depthw(depthw), float32 depthw_bounds(depthw,axis_nbounds), float32 vovecrtz(time_counter,depthw,y,x), float64 time_centered(time_counter), float64 time_centered_bounds(time_counter,axis_nbo